# X-ray diffraction (XRD) interactive plot

*Author: Enze Chen, PhD student at the University of California, Berkeley*

This is an interactive notebook for playing around with some experimental parameters ($a$, $\lambda$, $T$, etc.) and observing the effect on the resulting XRD spectra. I find XRD to be a particularly beautiful subject and I couldn't find any similar visualizations online. I hope this interactive demo will help you learn the general _trends_ associated with powder XRD spectra. Please don't hesitate to reach out if you have questions or ideas to contribute.

## Acknowledgements

I would like to thank Prof. Andrew Minor for teaching MATSCI 204: Materials Characterization and my advisor Prof. Mark Asta for his unwavering encouragement. Interactivity is enabled with the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) library.

## Important equations

The most important equation is Bragg's law, given by 

$$n\lambda = 2d \sin(\theta)$$

where $n$ is the order (typically $1$), $\lambda$ is the wavelength, $d$ is the interplanar spacing, and $\theta$ is the Bragg angle. In creating the interactive widget, we will need to solve for $\theta$ as follows:

$$ \lambda = 2d \sin(\theta) \longrightarrow \theta = \sin^{-1} \left( \frac{\lambda}{2d} \right), \quad d = \frac{a}{\sqrt{h^2 + k^2 + l^2}} $$

where $h,k,l$ are the miller indices of the diffracting plane and $a$ is the lattice constant. 

Another important equation is the intensity, given by

$$ I = |F|^2 \times P \times m \times L \times A \times T $$

where
* $F$ is the structure factor.
* $P$ is the polarization factor.
* $m$ is the multiplicity factor.
* $L$ is the Lorentz factor.
* $A$ is the absorption factor.
* $T$ is the temperature factor.

Furthermore, size effects can be included through the Scherrer equation, given by 

$$ t = \frac{K\lambda}{\beta \cos(\theta)} $$ 

where $t$ is the thickness, $K \sim 0.9$, and $\beta$ is the FWHM of the peak in radians.

For more information, please reference [Elements of X-Ray Diffraction (3rd) - Cullity and Stock](https://www.pearson.com/us/higher-education/program/Cullity-Elements-of-X-Ray-Diffraction-3rd-Edition/PGM113710.html).

### Assumptions I've taken great liberties with:

* For the structure factor, I greatly oversimplified the construction of the atomic scattering factor and selected some numbers that more or less came from the data for iron since it has both BCC and FCC structures.
* I also combined part of the Temperature factor into the structure factor.
* I combined the Lorentz and Polarization factors, as is commonly done in the literature.
* Like any good computationalist (jk), I threw out most constant factors.
* I ignored the absorption factor since it is more or less independent of $\theta$.
* I used a $\sqrt[3]{\theta}$ term to approximate the thermal background's general shape. I don't know the actual dependence, if there is one.
* It only models FCC and BCC structures with single-atom bases. I think it's enough to get the trends, as calculations for specific materials can be found online (e.g. at [The Materials Project](https://materialsproject.org/)).
* I used a Gaussian distribution to model each peak so that I could capture crystallite size effects using the Scherrer equation. Peaks in general are not Gaussian.
* It doesn't have great safeguards against errors, such as invalid `arcsin` arguments (typically if $\lambda$ is large and $d$ is small). Please be gentle. ❤

## Python package imports

These are all the required Python packages.

In [1]:
# General libraries
import itertools

# Scientific computing libraries
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity libraries
from ipywidgets import interact_manual, fixed, \
                       IntSlider, FloatSlider, FloatLogSlider, RadioButtons, \
                       Button, Layout

## Useful parameters

Just some helpful parameters I can quickly query.

In [2]:
int_to_hkl = {1:'(100)', 2:'(110)', 3:'(111)', 4:'(200)', 5:'(210)', 6:'(211)', 
              8:'(220)', 9:'(221,(300)', 10:'(310)', 11:'(311)', 12:'(222)', 13:'(320)', 14:'(321)',
              16:'(400)', 17:'(410),(322)', 18:'(411),(330)', 19:'(331)', 20:'(420)'}
fcc_ratios = np.array([1, 4/3, 8/3, 11/3, 12/3, 16/3, 19/3, 20/3])
bcc_ratios = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
dc_ratios = np.array([1, 8/3, 11/3, 16/3, 19/3])

## Widget function

This function handles all the calculations.

In [21]:
def plot_XRD(a, wavelength, cell_type, thickness, T=0, K=0.94):

    # Determine planes from the structure factor
    if cell_type is 'FCC':
        planes = [[1,1,1], [2,0,0], [2,2,0], [3,1,1], [2,2,2], [4,0,0]]
        planes_str = ['(111)', '(200)', '(220)', '(311)', '(222)', '(400)']
    elif cell_type is 'BCC':
        planes = [[1,1,0], [2,0,0], [2,1,1], [2,2,0], [3,1,0], [2,2,2]]
        planes_str = ['(110)', '(200)', '(211)', '(220)', '(310)', '(222)']
    else:
        print('Cell type not yet supported.')
        raise ValueError

    # Convert planes to theta values (see equation above)
    a += 1e-5 * T  # thermal expansion estimate
    s_vals = np.array([np.linalg.norm(p) for p in planes])
    theta = np.arcsin(np.divide(wavelength/2, np.divide(a, s_vals)))
    two_theta = 2 * np.degrees(theta)

    # Scherrer equation calculations
    beta = np.degrees(K * wavelength / thickness * np.divide(1, np.cos(theta)))
    sigma = beta / 2.355

    # Structure-Temperature factor
    s = np.sin(theta) / wavelength
    f = 26 - 41.8 * 7 * np.multiply(s**2, np.exp(-(s)**2))
    if cell_type is 'FCC':
        F = 4*f
    elif cell_type is 'BCC':
        F = 2*f

    # Multiplicity factor
    mult = [2**np.count_nonzero(p) * sum(1 for _ in set(itertools.permutations(p))) for p in planes]

    # Lorentz-Polarization factor
    P = np.divide(1 + np.cos(2 * theta)**2, np.multiply(np.sin(theta)**2, np.cos(theta)))

    # Final intensity (numpy hacks)
    factors = [F, mult, P]
    I = np.prod(np.vstack(factors), axis=0)
    
    plt.rcParams.update({'figure.figsize':(13,5), 'font.size':22})
    xmin, xmax = (20, 160)
    fig, ax = plt.subplots()

    x = np.linspace(xmin, xmax, int(1e4))
    thermal_diffuse = T/2 * np.cbrt(x)    # THIS FUNCTIONAL DEPENDENCE IS NOT REAL!!!
                                          # Chosen just to get general shape of background
    all_curves = []
    for i in range(len(sigma)):
        y = stats.norm.pdf(x, two_theta[i], sigma[i])
        normed_curve = y / max(y) * I[i]
        max_ind = normed_curve.argmax()
        ax.annotate(s=planes_str[i], xy=(x[max_ind], normed_curve[max_ind] + thermal_diffuse[max_ind]))
        all_curves.append(normed_curve)
    final_curve = np.max(all_curves, axis=0) + thermal_diffuse
    plt.plot(x, final_curve, c='C0', lw=3)

    for side in ['top', 'right']:
        ax.spines[side].set_visible(False)
    for side in ['left', 'bottom']:
        ax.spines[side].set_linewidth(2)
    ax.tick_params(left=False, labelleft=False, direction='in', length=10, width=2)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(0, 1.05*ax.get_ylim()[1])
    ax.set_xlabel(r'$2\theta\ (^{\circ})$')
    ax.set_ylabel('Intensity (a.u.)')
    plt.show()

I've created each slider individually for readability and customization.

In [22]:
b = Button(description='A button', layout=Layout(width='400px', height='40px'))

a_widget = FloatSlider(value=0.352, min=0.25, max=0.45, step=0.001, description='Lattice constant (nm)', \
                       readout_format='.3f', style={'description_width':'150px'}, layout=b.layout)

w_widget = FloatSlider(value=0.154, min=0.070, max=0.200, step=0.001, description='X-ray wavelength (nm)',\
                       readout_format='.3f', style={'description_width':'150px'}, layout=b.layout)

c_widget = RadioButtons(options=['FCC', 'BCC'], description='Crystal structure',\
                        style={'description_width':'150px'}, layout=b.layout)

t_widget = FloatLogSlider(value=10, base=10, min=0, max=3, step=0.1, description='Crystallite size (nm)',\
                          readout_format='d', style={'description_width':'150px'}, layout=b.layout)

T_widget = IntSlider(value=298, min=0, max=1000, step=1, description='Temperature (K)',\
                     readout_format='d', style={'description_width':'150px'}, layout=b.layout)

In [23]:
interact_manual(plot_XRD, a=a_widget, wavelength=w_widget, cell_type=c_widget, 
                          thickness=t_widget, T=T_widget, K=fixed(0.94));

interactive(children=(FloatSlider(value=0.352, description='Lattice constant (nm)', layout=Layout(height='40px…